In [12]:
using Test

In [13]:
f = readlines("day16.input");

In [14]:
function parse_input(input)
    flows = Dict{String, Int64}()
    edges = Dict{String, Vector{String}}()
    for line in input
        name_re = r"Valve ([A-Z][A-Z]) has flow rate=\d+; tunnels? leads? to valves? .*"
        flow_re = r"Valve [A-Z][A-Z] has flow rate=(\d+); tunnels? leads? to valves? .*"
        edge_re = r"Valve [A-Z][A-Z] has flow rate=\d+; tunnels? leads? to valves? (.*)"
        name = first(match(name_re, line).captures)
        flows[name] = parse(Int64, first(match(flow_re, line).captures))
        edges[name] = [strip(x) for x in (split(first(match(edge_re, line).captures), ","))]
    end
    flows, edges
end

parse_input (generic function with 1 method)

In [15]:
function is_opened(open, pos)
    pos in open
end

is_opened (generic function with 1 method)

In [16]:
function open_valve(open, pos)
    new_open = copy(open)
    push!(new_open, pos)
    return new_open
end

open_valve (generic function with 1 method)

In [17]:
function solve(pos, time, open, adjacent, flows, states)
    stored = get(states, (pos, time, open), -1)
    if stored > -1
        return stored
    end

    if time == 0
        if pos == "AA"
            return 0
        end
        return -1000
    end
    
    scores = []
    
    # Open valve
    if flows[pos] > 0 && !is_opened(open, pos)
        score = (30 - time) * flows[pos] + solve(pos, time-1, open_valve(open, pos), adjacent, flows, states)
        push!(scores, score)
    end
    
    # Move
    for adj in adjacent[pos]
        score = solve(adj, time-1, open, adjacent, flows, states)
        push!(scores, score)
    end
    
    result = maximum(scores)
    states[(pos, time, open)] = result
    return result
end

solve (generic function with 1 method)

In [29]:
function solve_part_1(input)
    flows, edges = parse_input(input)

    states = Dict{Tuple{String, Int64, Set{String}}, Int64}()

    scores = []
    for location in keys(flows)
        score = solve(location, 30, Set(), edges, flows, states)
        push!(scores, score)
    end
    

    println("We look through $(length(states)) states. Type was $(typeof(states)).")

    maximum(scores)
end

solve_part_1 (generic function with 1 method)

In [30]:
@test solve_part_1(String.(split("Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II", "\n"))) == 1651


We look through 13113 states. Type was Dict{Tuple{String, Int64, Set{String}}, Int64}.


Test Passed

In [31]:
solve_part_1(f)

We look through 2714888 states. Type was Dict{Tuple{String, Int64, Set{String}}, Int64}.


2056